In [1]:
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/10/05 03:19:15 WARN Utils: Your hostname, Runyus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.12 instead (on interface en0)
22/10/05 03:19:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 03:19:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/05 03:19:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/05 03:19:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/05 03:19:16 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
consumer_fraud_sdf = read_file(spark, 'consumer_fraud.parquet', '../data/curated/')

|> Loading File...


|> Loading Finished!


-RECORD 0---------------------------------------------------------------------------------------------
 user_id           | 9010                                                                             
 consumer_id       | 1053884                                                                          
 take_rate         | 5.73                                                                             
 tagIndex          | 0.0                                                                              
 tag_vec           | (24,[0],[1.0])                                                                   
 count             | 1                                                                                
 dollar_average    | 2.2484627193298805                                                               
 dollar_min        | 2.2484627193298805                                                               
 dollar_max        | 2.2484627193298805                                  

In [5]:
merchant_fraud_sdf = read_file(spark, 'merchant_fraud.parquet', '../data/curated/')

|> Loading File...
|> Loading Finished!
-RECORD 0---------------------------------------------------------------------------------------------
 merchant_abn      | 43719937438                                                                      
 take_rate         | 5.01                                                                             
 tagIndex          | 4.0                                                                              
 tag_vec           | (24,[4],[1.0])                                                                   
 count             | 11                                                                               
 dollar_average    | 76.74733459026552                                                                
 dollar_min        | 27.621111402431843                                                               
 dollar_max        | 167.7734627758464                                                                
 order_year        | 2021        

In [6]:
trans_df = read_file(spark, 'changed_data', '../data/curated/')

|> Loading File...
|> Loading Finished!
-RECORD 0--------------------------------------------------------------------------
 user_id           | 8972                                                          
 merchant_abn      | 59658193201                                                   
 dollar_value      | 182.16129389048888                                            
 order_id          | 5a2112b5-6a59-4ceb-95a5-655fa5915c7a                          
 merchant_name     | Id Ante Institute                                             
 tags              | cable, satellite, and other pay television and radio services 
 take_rate         | 6.16                                                          
 type              | a                                                             
 postcode          | 5311                                                          
 consumer_id       | 1115127                                                       
 name              | Mr. Bradley Moo

In [7]:
from pyspark.sql.functions import year, month, dayofmonth

In [8]:
trans_df = (trans_df
               .withColumn('order_year', year(trans_df.order_datetime))
               .withColumn('order_month', month(trans_df.order_datetime))
               .withColumn('order_day', dayofmonth(trans_df.order_datetime))
               ).drop('order_datetime')

In [9]:
trans_df.show(5)

22/10/05 03:19:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+---------------+----------+------------+-----------------+-----------+---------+--------+---------------+----------+-----------+---------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|postcode|consumer_id|             name|             address|state|gender|POA_CODE21|       population|New cases / day|     month|total_retail|__index_level_0__|genderIndex|typeIndex|tagIndex|        tag_vec|order_year|order_month|order_day|
+-------+------------+------------------+--------------------+--------------------+-----------------

### Three categories based on the average value of orders

- Merchant Classification

Classification of shops according to their average sales level

In [10]:
from pyspark.ml.feature import QuantileDiscretizer

In [11]:
dqis = QuantileDiscretizer(numBuckets=3, inputCol='dollar_average', outputCol='dollar_stage', relativeError=0.001)

In [12]:
model = dqis.fit(merchant_fraud_sdf)

In [13]:
merchant_fraud_sdf = model.transform(merchant_fraud_sdf)

In [14]:
merchant_fraud_sdf.show()

+------------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+------------------+------------+
|merchant_abn|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max|order_year|order_month|order_day|            features| fraud_probability|dollar_stage|
+------------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+------------------+------------+
| 43719937438|     5.01|     4.0| (24,[4],[1.0])|   11| 76.74733459026552|27.621111402431843| 167.7734627758464|      2021|         11|       26|(34,[0,1,2,7,27,2...| 93.31059770507272|         0.0|
| 72553304202|     5.04|     4.0| (24,[4],[1.0])|   30|53.323260095837426| 11.07002991392789|146.32500496249799|      2021|         11|       26|(34,[0,1,2,7,27,2...| 88.55146900735417|         0.0|
| 286

- Four categories of merchants based on number of purchases

In [15]:
dqis2 = QuantileDiscretizer(numBuckets=4, inputCol='count', outputCol='count_stage', relativeError=0.001)

In [16]:
model2 = dqis2.fit(merchant_fraud_sdf)

In [17]:
merchant_fraud_sdf = model2.transform(merchant_fraud_sdf)

In [18]:
merchant_fraud_sdf.show()

+------------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+------------------+------------+-----------+
|merchant_abn|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max|order_year|order_month|order_day|            features| fraud_probability|dollar_stage|count_stage|
+------------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+------------------+------------+-----------+
| 43719937438|     5.01|     4.0| (24,[4],[1.0])|   11| 76.74733459026552|27.621111402431843| 167.7734627758464|      2021|         11|       26|(34,[0,1,2,7,27,2...| 93.31059770507272|         0.0|        3.0|
| 72553304202|     5.04|     4.0| (24,[4],[1.0])|   30|53.323260095837426| 11.07002991392789|146.32500496249799|      2021|         11|       26|(34,[0,1,2,

- Join with transaction data

In [19]:
consumer_fraud = consumer_fraud_sdf.select('user_id', 'order_year', 'order_month', 'order_day', 'fraud_probability')

In [20]:
consumer_fraud = consumer_fraud.withColumnRenamed('fraud_probability', 'consumer_fraud_probability')

In [21]:
consumer_fraud.show(5)

+-------+----------+-----------+---------+--------------------------+
|user_id|order_year|order_month|order_day|consumer_fraud_probability|
+-------+----------+-----------+---------+--------------------------+
|   9010|      2021|         11|       26|         11.53696660751666|
|   8942|      2021|         11|       26|         11.56013830129541|
|  11795|      2021|         11|       26|        13.605123123049225|
|  10548|      2021|         11|       26|         13.67580908072864|
|  10120|      2021|         11|       26|        13.930895430112514|
+-------+----------+-----------+---------+--------------------------+
only showing top 5 rows



In [22]:
sdf = trans_df.join(consumer_fraud, on=['user_id', 'order_year', 'order_month', 'order_day'], how = 'left')

In [23]:
sdf.show()

+-------+----------+-----------+---------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+------------------+--------------------+-----+-----------+----------+-----------------+---------------+----------+------------+-----------------+-----------+---------+--------+---------------+--------------------------+
|user_id|order_year|order_month|order_day|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|postcode|consumer_id|              name|             address|state|     gender|POA_CODE21|       population|New cases / day|     month|total_retail|__index_level_0__|genderIndex|typeIndex|tagIndex|        tag_vec|consumer_fraud_probability|
+-------+----------+-----------+---------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+------------------+--------------------+-

In [24]:
merchant_fraud = merchant_fraud_sdf.select('merchant_abn', 'order_year', 'order_month', 'order_day', 'dollar_stage','count_stage','count','fraud_probability')

In [25]:
merchant_fraud = merchant_fraud.withColumnRenamed('fraud_probability', 'merchant_fraud_probability')

In [26]:
merchant_fraud.show(5)

+------------+----------+-----------+---------+------------+-----------+-----+--------------------------+
|merchant_abn|order_year|order_month|order_day|dollar_stage|count_stage|count|merchant_fraud_probability|
+------------+----------+-----------+---------+------------+-----------+-----+--------------------------+
| 43719937438|      2021|         11|       26|         0.0|        3.0|   11|         93.31059770507272|
| 72553304202|      2021|         11|       26|         0.0|        3.0|   30|         88.55146900735417|
| 28690231799|      2021|         11|       26|         1.0|        3.0|   24|         50.78452165739873|
| 45785438987|      2021|         11|       26|         2.0|        2.0|    4|         44.42391595533991|
| 41485392864|      2021|         11|       26|         1.0|        2.0|    3|         61.59168677152047|
+------------+----------+-----------+---------+------------+-----------+-----+--------------------------+
only showing top 5 rows



In [27]:
sdf = sdf.join(merchant_fraud, on=['merchant_abn', 'order_year', 'order_month', 'order_day'], how = 'left')

In [28]:
sdf.show()

+------------+----------+-----------+---------+-------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+-----------+----------+-----------------+---------------+----------+------------+-----------------+-----------+---------+--------+---------------+--------------------------+------------+-----------+-----+--------------------------+
|merchant_abn|order_year|order_month|order_day|user_id|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|postcode|consumer_id|             name|             address|state|     gender|POA_CODE21|       population|New cases / day|     month|total_retail|__index_level_0__|genderIndex|typeIndex|tagIndex|        tag_vec|consumer_fraud_probability|dollar_stage|count_stage|count|merchant_fraud_probability|
+------------+----------+-----------+---------+-------+------------------+--------------------+-----

- Rank Model

In [29]:
sdf = sdf.withColumn('average_probability', ((sdf.consumer_fraud_probability + sdf.merchant_fraud_probability)/2))

In [30]:
sdf = sdf.filter(sdf.average_probability <= 80)

In [31]:
sdf.show(5)

+------------+----------+-----------+---------+-------+------------------+--------------------+-------------+--------------------+---------+----+--------+-----------+--------------+--------------------+-----+-----------+----------+-----------------+---------------+----------+------------+-----------------+-----------+---------+--------+---------------+--------------------------+------------+-----------+-----+--------------------------+-------------------+
|merchant_abn|order_year|order_month|order_day|user_id|      dollar_value|            order_id|merchant_name|                tags|take_rate|type|postcode|consumer_id|          name|             address|state|     gender|POA_CODE21|       population|New cases / day|     month|total_retail|__index_level_0__|genderIndex|typeIndex|tagIndex|        tag_vec|consumer_fraud_probability|dollar_stage|count_stage|count|merchant_fraud_probability|average_probability|
+------------+----------+-----------+---------+-------+------------------+------

- Top10 by dollar_stage

In [32]:
merchant_group = sdf.groupBy('merchant_abn','merchant_name', 'tags', 'take_rate','type', 'dollar_stage','average_probability').agg(F.count("*").alias("count"))

In [33]:
merchant_group.show(10)

+------------+--------------------+--------------------+---------+----+------------+-------------------+-----+
|merchant_abn|       merchant_name|                tags|take_rate|type|dollar_stage|average_probability|count|
+------------+--------------------+--------------------+---------+----+------------+-------------------+-----+
| 10023283211|       Felis Limited|furniture, home f...|     0.18|   e|         1.0|  40.34654451636561|    1|
| 10192359162| Enim Condimentum PC|music shops - mus...|     6.33|   a|         0.0|  23.99760945328933|    1|
| 10206519221|       Fusce Company|gift, card, novel...|     6.34|   a|         0.0| 49.914418023701614|    1|
| 10206519221|       Fusce Company|gift, card, novel...|     6.34|   a|         0.0|  46.10948378688374|    1|
| 10206519221|       Fusce Company|gift, card, novel...|     6.34|   a|         0.0|  46.30293406769988|    1|
| 10264435225|    Ipsum Primis Ltd|watch, clock, and...|     2.39|   c|         1.0|  38.29529137902216|    1|
|

In [34]:
tag1 = merchant_group.filter(merchant_group.dollar_stage == 0.0).orderBy('average_probability', ascending=False)

In [35]:
tag1.show(10)

+------------+--------------------+--------------------+---------+----+------------+-------------------+-----+
|merchant_abn|       merchant_name|                tags|take_rate|type|dollar_stage|average_probability|count|
+------------+--------------------+--------------------+---------+----+------------+-------------------+-----+
| 77313075219|     Sed Sem Limited|cable, satellite,...|     4.21|   b|         0.0|  77.54310334619197|    1|
| 78042516048|    Est Ac Institute|gift, card, novel...|     1.89|   c|         0.0|  77.49506145274063|    2|
| 28748147493|     Neque Institute|gift, card, novel...|     2.51|   c|         0.0|   76.3554509906246|    1|
| 24598653065|Et Rutrum Foundation|cable, satellite,...|     2.17|   c|         0.0|  76.24724961759671|    1|
| 17739089622|   Auctor Quis Corp.|cable, satellite,...|     5.01|   b|         0.0|   75.7739223225409|    2|
| 88124569339|Non Vestibulum Corp.|cable, satellite,...|     3.58|   b|         0.0|  75.72645994098582|    1|
|

In [36]:
tag2 = merchant_group.filter(merchant_group.dollar_stage == 1.0).orderBy('average_probability', ascending=False)

In [37]:
tag2.show(10)

+------------+--------------------+--------------------+---------+----+------------+-------------------+-----+
|merchant_abn|       merchant_name|                tags|take_rate|type|dollar_stage|average_probability|count|
+------------+--------------------+--------------------+---------+----+------------+-------------------+-----+
| 20885454195|Pharetra Ut Indus...|cable, satellite,...|     4.94|   b|         1.0|  78.47401844213994|    2|
| 68501926042|         Dolor Corp.|cable, satellite,...|     5.78|   a|         1.0|  78.42634847312729|    2|
| 34822634769|Dolor Egestas Rho...|cable, satellite,...|     0.71|   d|         1.0|  78.19261611714717|    1|
| 32361057556|Orci In Consequat...|gift, card, novel...|     6.61|   a|         1.0|   76.7183049333812|    2|
| 40252040480|Luctus Felis Puru...|cable, satellite,...|     5.77|   a|         1.0|  76.40486046267188|    2|
| 16286338734|            Vitae PC|gift, card, novel...|      2.6|   c|         1.0|  75.36473592127834|    1|
|

In [38]:
tag3 = merchant_group.filter(merchant_group.dollar_stage == 2.0).orderBy('average_probability', ascending=False)

In [39]:
tag3.show(10)

+------------+--------------------+--------------------+---------+----+------------+-------------------+-----+
|merchant_abn|       merchant_name|                tags|take_rate|type|dollar_stage|average_probability|count|
+------------+--------------------+--------------------+---------+----+------------+-------------------+-----+
| 92065881715|Quam Elementum Corp.|opticians, optica...|      4.2|   b|         2.0|  76.41852161366717|    2|
| 12173100693|Quam Quis Diam Fo...|gift, card, novel...|     5.65|   a|         2.0|  74.28253374570204|    1|
| 42368556376|    At Iaculis Corp.|florists supplies...|     2.26|   c|         2.0|  72.87785900052893|    2|
| 70052129860|Donec Tempus Lore...|cable, satellite,...|     3.37|   b|         2.0|  72.79260740925349|    1|
| 81219314324|    Faucibus Leo Ltd|gift, card, novel...|     6.35|   a|         2.0|   72.4550320083157|    1|
| 86889657711|Accumsan Corporation|jewelry, watch, c...|     4.65|   b|         2.0|  72.38815946111743|    2|
|

- Top10 by count stage

In [43]:
merchant_group = sdf.groupBy('merchant_abn','merchant_name', 'tags', 'take_rate','type', 'count_stage','average_probability').agg(F.sum("count").alias("count"))

In [44]:
merchant_group.show(10)

+------------+--------------------+--------------------+---------+----+-----------+-------------------+-----+
|merchant_abn|       merchant_name|                tags|take_rate|type|count_stage|average_probability|count|
+------------+--------------------+--------------------+---------+----+-----------+-------------------+-----+
| 10023283211|       Felis Limited|furniture, home f...|     0.18|   e|        2.0| 41.057510183527825|    5|
| 10023283211|       Felis Limited|furniture, home f...|     0.18|   e|        2.0|  39.05099765263685|    6|
| 10023283211|       Felis Limited|furniture, home f...|     0.18|   e|        2.0|  38.39515796351304|    4|
| 10206519221|       Fusce Company|gift, card, novel...|     6.34|   a|        3.0|  50.64524173828238|   14|
| 10206519221|       Fusce Company|gift, card, novel...|     6.34|   a|        3.0|  46.05550151973796|   25|
| 10206519221|       Fusce Company|gift, card, novel...|     6.34|   a|        3.0|  44.91414488166947|   21|
| 10264435

In [56]:
level0 = merchant_group.filter(merchant_group.count_stage == 0.0).orderBy('average_probability', ascending=False)

In [57]:
level0.show(10)

+------------+-------------+----+---------+----+-----------+-------------------+-----+
|merchant_abn|merchant_name|tags|take_rate|type|count_stage|average_probability|count|
+------------+-------------+----+---------+----+-----------+-------------------+-----+
+------------+-------------+----+---------+----+-----------+-------------------+-----+



In [47]:
level1 = merchant_group.filter(merchant_group.count_stage == 1.0).orderBy('average_probability', ascending=False)

In [51]:
level1.show(10)

+------------+--------------------+--------------------+---------+----+-----------+-------------------+-----+
|merchant_abn|       merchant_name|                tags|take_rate|type|count_stage|average_probability|count|
+------------+--------------------+--------------------+---------+----+-----------+-------------------+-----+
| 79657127260|Consectetuer Ipsu...|cable, satellite,...|     6.73|   a|        1.0|  74.83890442674146|    1|
| 86889657711|Accumsan Corporation|jewelry, watch, c...|     4.65|   b|        1.0|  72.38815946111743|    2|
| 51425051982|     Sem Corporation|antique shops - s...|     5.71|   a|        1.0|  71.25574558966503|    2|
| 97884414539|      Ut Corporation|antique shops - s...|     6.82|   a|        1.0|  71.22456458785155|    2|
| 89109402284|Interdum Feugiat ...|antique shops - s...|     3.87|   b|        1.0|  70.15915435753925|    2|
| 38918664617| Faucibus Foundation|jewelry, watch, c...|     3.59|   b|        1.0|  70.08014123062867|    2|
| 79100970

In [48]:
level2 = merchant_group.filter(merchant_group.count_stage == 2.0).orderBy('average_probability', ascending=False)

In [52]:
level2.show(10)

+------------+--------------------+--------------------+---------+----+-----------+-------------------+-----+
|merchant_abn|       merchant_name|                tags|take_rate|type|count_stage|average_probability|count|
+------------+--------------------+--------------------+---------+----+-----------+-------------------+-----+
| 77313075219|     Sed Sem Limited|cable, satellite,...|     4.21|   b|        2.0|  77.54310334619197|    4|
| 28748147493|     Neque Institute|gift, card, novel...|     2.51|   c|        2.0|   76.3554509906246|    3|
| 24598653065|Et Rutrum Foundation|cable, satellite,...|     2.17|   c|        2.0|  76.24724961759671|    5|
| 98545158925|Dictum Eleifend I...|cable, satellite,...|     2.13|   c|        2.0|  74.90710869106545|    6|
| 33388024878|    Semper Institute|gift, card, novel...|     4.27|   b|        2.0|  74.55944481723077|    4|
| 42368556376|    At Iaculis Corp.|florists supplies...|     2.26|   c|        2.0|  72.87785900052893|   12|
| 45339304

In [49]:
level3 = merchant_group.filter(merchant_group.count_stage == 3.0).orderBy('average_probability', ascending=False)

In [53]:
level3.show(10)

+------------+--------------------+--------------------+---------+----+-----------+-------------------+-----+
|merchant_abn|       merchant_name|                tags|take_rate|type|count_stage|average_probability|count|
+------------+--------------------+--------------------+---------+----+-----------+-------------------+-----+
| 20885454195|Pharetra Ut Indus...|cable, satellite,...|     4.94|   b|        3.0|  78.47401844213994|   54|
| 68501926042|         Dolor Corp.|cable, satellite,...|     5.78|   a|        3.0|  78.42634847312729|   62|
| 34822634769|Dolor Egestas Rho...|cable, satellite,...|     0.71|   d|        3.0|  78.19261611714717|   24|
| 78042516048|    Est Ac Institute|gift, card, novel...|     1.89|   c|        3.0|  77.49506145274063|   20|
| 32361057556|Orci In Consequat...|gift, card, novel...|     6.61|   a|        3.0|   76.7183049333812|  264|
| 92065881715|Quam Elementum Corp.|opticians, optica...|      4.2|   b|        3.0|  76.41852161366717|   26|
| 40252040

## Three categories based on life cycle

In [40]:
No_clear_periodicity = ['antique shops - sales, repairs, and restoration services',
                         'bicycle shops - sales and service',
                         'equipment, tool, furniture, and appliance rent al and leasing',
                         'gift, card, novelty, and souvenir shops',
                         'health and beauty spas',
                         'hobby, toy and game shops',
                         'jewelry, watch, clock, and silverware shops',
                         'shoe shops',
                         'lawn and garden supply outlets, including nurseries',
                         'digital goods: books, movies, music'
                         ]

In [41]:
once_used = [
    'art dealers and galleries',
    'artist supply and craft shops',
    'computer programming , data processing, and integrated systems design services',
    'computers, computer peripheral equipment, and software',
    'motor vehicle supplies and new parts',
    'furniture, home furnishings and equipment shops, and manufacturers, except appliances',
    'music shops - musical instruments, pianos, and sheet music',
    'tent and awning shops'
]

In [42]:
periodicity_product = [
    'books, periodicals, and newspapers',
    'florists supplies, nursery stock, and flowers',
    'lawn and garden supply outlets, including nurseries',
    
]